<a href="https://colab.research.google.com/github/junhochoi-git/open-api-to-csv-convertor/blob/main/openApi_to_csv_convertor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 오픈 API로 특정 URL 을 형식에 맞게 요청하여 데이터를 응답받을때, 응답받은 데이터를 CSV파일로 변환하는 코드를 작성함

In [ ]:
# sample Open API links
# https://www.data.go.kr/iim/api/selectAPIAcountView.do

import requests
import pandas as pd

month = ["01","02","03","04","05","06","07","08","09","10","11","12"]

# -- 특정 달 데이터만 csv파일로 만든다 
def extract_data(endpoint,useYear,m_idx,sx): #1. endpoint, Year, from Jan to Dec, and "여자" or "남자"의 성별 
    
    # 요청 파라미터
    service_key = "WRITE_YOUR_OWN_SERVICE_KEY_HERE" # 제가 요청한 서비스키
    numOfRows = 50
    pageNo = 0
    viewType = "json"
    useYymm="2020" 
    sxClNm=sx
    # 요청 폼

    page_cnt = 1
    result = [] # 각 json 데이터를 dict형들의 리스트로 담고 있는 전체 리스트

    useYymm += month[m_idx] # 최초에 "202001"로 설정
    
    while m_idx < 12: 

        pageNo = page_cnt
        req = f"{endpoint}?serviceKey={service_key}&numOfRows={numOfRows}&pageNo={pageNo}&useYymm={useYymm}&sxClNm={sxClNm}"
        req += f"&viewType={viewType}"
        
        # API 호출
        resp = requests.get(req)
        temp = resp.json()
        
        if not temp['response']['body']['items']: # 해당 달의 데이터를 모두 가져왔다면, month 증가, page_cnt = 1 부터 다시 요청 
            page_cnt = 1
            # m_idx += 1 # 01월 ~ 12월 까지 정보를 한번에 얻고 싶을땐, 아래 break 문은 주석처리하고 m_idx는 주석해제
            useYymm = "2020" + month[m_idx]
            # 현재 한달치 자료만 추출 
            break # -- 한번에 01월 부터 12월 까지 추출 및 csv 파일 변환을 원한다면 break 해제하기
        
        result += temp['response']['body']['items']
        page_cnt += 1
        # print(f"{page_cnt}p , {m_idx}")
    return result

def save_csv(_list, sx, year,month):
    df_items = pd.DataFrame(_list)
    df_items.to_csv(f"맞춤형복지_신용카드_소비내역_정보_{sx}성_{year}_{month}.csv", encoding="utf-8-sig")

endpoint = "http://apis.data.go.kr/B552640/GwpCardUseDtlInfoService/selectListUseDtl"


for i in range(0,11):   
    result = extract_data(endpoint,"2020",i,"여")
    print(len(result))
    save_csv(result, "여",2020,month[i])

JSONDecodeError: ignored

### 하루 트래픽을 초과하면 응답이 제대로 되지 않아 해당 코드가 작동을 안함을 주의